In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pwd

/content


# **Import libraries**

In [0]:
import json
from collections import defaultdict
import gensim
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import os
import glob
nlp = spacy.load('en', disable=['parser', 'ner'])

# **Doc2vec Implementation**

Preprocessing data through Doc2vec 

In [0]:
class textProcess:

    def preprocessText(self, text):

        text_gensim = ' '.join(gensim.utils.simple_preprocess(text, deacc=True))
        text_obj = nlp(text_gensim)
        text_lemma = [token.lemma_ for token in text_obj]
        return ' '.join(text_lemma)

    def createVocab(self, corpus, ngram_range=(1, 1)):

        countVec = CountVectorizer(stop_words='english', ngram_range=ngram_range,
                                   analyzer='word', strip_accents='unicode')
        countVocab = countVec.fit(corpus)
        return countVocab.vocabulary_

    def generateDoc2Vec(self, articles, is_dbow=False, is_hs=False, embedding_dim=100, max_epochs=10, win=5, min_c=1, neg=5):

        if is_dbow == True and is_hs == False:
                d2v_model_name = "PV-DBOW"
        elif is_dbow == False and is_hs == True:
                d2v_model_name = "PV-DM with hierarchical softmax"
        elif is_dbow == False and is_hs == False:
                d2v_model_name = "PV-DM with negative sampling"
        docTrain = []
        for artl in articles:
          docTrain.append(TaggedDocument(artl['words'], [artl['paper_id']]))
        if is_dbow:
                d2v_model = Doc2Vec(vector_size=embedding_dim, alpha=0.025, min_alpha=0.00025, window=win, min_count=min_c, workers=5, dm=0, dbow_words=0)
        elif is_hs and not is_dbow:
                d2v_model = Doc2Vec(vector_size=embedding_dim, alpha=0.025, min_alpha=0.00025, window=win, min_count=min_c, workers=5, dm=1,  hs=1, negative=neg, dm_mean=1, dm_concat=0)
        elif not is_hs and not is_dbow:
                d2v_model = Doc2Vec(vector_size=embedding_dim, alpha=0.025, min_alpha=0.00025, window=win, min_count=min_c, workers=5, dm=1,  hs=0, negative=neg, dm_mean=1, dm_concat=0)
        d2v_model.build_vocab(docTrain)
        print("Training Doc2Vec ({}) model on a corpus of {} articles".format(d2v_model_name, d2v_model.corpus_count))
        d2v_model.train(docTrain, total_examples=d2v_model.corpus_count, epochs=max_epochs)
        print("{} dimensional document vectors generated using {}".format(embedding_dim, d2v_model_name))
        d2v_model.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)
        return d2v_model

# **Doc2Vec outputs**

Run doc2vec on processed data

In [0]:
data_path = './drive/My Drive/CS235Project/DF_Outputs/DF_text_processed_V2.csv'
df_prcsd=pd.read_csv(data_path)  

In [0]:
df_prcsd['words']=df_prcsd['processed_body_text'].apply(lambda x:list(gensim.utils.simple_tokenize(x)))

In [0]:
textproc = textProcess()
arxiv_articles=df_prcsd.to_dict(orient='records')
PV_DBOW_arxiv = textproc.generateDoc2Vec(arxiv_articles, is_dbow=True)

Training Doc2Vec (PV-DBOW) model on a corpus of 10000 articles
100 dimensional document vectors generated using PV-DBOW


In [0]:
## PV-DBOW
PV_DBOW_arxiv = textproc.generateDoc2Vec(arxiv_articles, is_dbow=True)

Training Doc2Vec (PV-DBOW) model on a corpus of 3487 articles
100 dimensional document vectors generated using PV-DBOW


In [0]:
## PV-DM
PV_DM_arxiv = textproc.generateDoc2Vec(arxiv_articles, is_dbow=False)

Training Doc2Vec (PV-DM with negative sampling) model on a corpus of 3487 articles
100 dimensional document vectors generated using PV-DM with negative sampling


Saving the results

In [0]:
output_file="./drive/My Drive/CS235Project/Doc2Vec/PV_DBOW_prcsd.doc2vec"
PV_DBOW_arxiv.save(output_file)
output_file2="./drive/My Drive/CS235Project/Doc2Vec/PV_DM_prcsd.doc2vec"
PV_DM_arxiv.save(output_file2)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
